In [1]:
import pickle
import pandas as pd
from pling_evaluate import write_json, generate_result, ArenaEvaluator
import numpy as np

# Modeling

> Data Load

In [2]:
with open('../0_data/mfl_col.pkl', 'rb') as f:
    mfl_col = pickle.load(f)

print(f'mfl_col 개수: {len(mfl_col)}')

song_len = 22798
print(f'song_len 개수: {song_len}')
# song = 22798, tag = 1868

q_test = pd.read_json('../0_data/q_test.json')
print(f'q_test 행 개수: {len(q_test)}')

with open('../0_data/train_onehot.pkl', 'rb') as f:
    train_onehot = pickle.load(f)

print(f'train_onehot.shape: {train_onehot.shape}')

with open('../0_data/q_test_onehot.pkl', 'rb') as f:
    q_test_onehot = pickle.load(f)

print(f'q_test_onehot.shape: {q_test_onehot.shape}')

mfl_col 개수: 24666
song_len 개수: 22798
q_test 행 개수: 11456
train_onehot.shape: (45824, 24666)
q_test_onehot.shape: (11456, 24666)


> Modeling

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Dropout, BatchNormalization, LeakyReLU
from tensorflow.keras.optimizers import Adam

> model_1
- input_dim = 24666 #len(mfl_col)
- hidden_dim = 64
- dropout = 0.2
- BatchNormalization(input_shape=(hidden_dim,))
- LeakyReLU(alpha=0.01)
- epoch = 10
- batch_size = 64
- loss_function = binary_crossentropy
- learning_rate = 0.0005
- optimizer = adam(learning_rate=learning_rate)

In [14]:
model_name = 'model_1' # 저장될 모델 이름

In [5]:
# model_1
input_dim = len(mfl_col)
hidden_dim = 64
dropout_rate = 0.2

input_layer = Input(shape=(input_dim,))
encoded = Dropout(0.2)(input_layer)
encoded = Dense(hidden_dim)(encoded)
encoded = BatchNormalization(input_shape=(hidden_dim,))(encoded)
encoded = LeakyReLU(alpha=0.01)(encoded)

decoded = Dense(input_dim, activation='sigmoid')(encoded)

model = Model(inputs=input_layer, outputs=decoded)

# 모델 요약 출력
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 24666)]           0         
                                                                 
 dropout_1 (Dropout)         (None, 24666)             0         
                                                                 
 dense_2 (Dense)             (None, 64)                1578688   
                                                                 
 batch_normalization_1 (Bat  (None, 64)                256       
 chNormalization)                                                
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 24666)             1603290   
                                                           

In [11]:
learning_rate = 0.0005
optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer,loss='binary_crossentropy')

In [ ]:
model.fit(train_onehot, train_onehot, epochs=10, batch_size=64, validation_split=0.2, verbose=1)

In [13]:
model.save(f'results/{model_name}.h5')

c:\Users\Playdata\anaconda3\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [15]:
model = tf.keras.models.load_model(f'results/{model_name}.h5')

In [16]:
predict_plist=model.predict(q_test_onehot)

358/358 [==============================] - 6s 15ms/step


> 결과 추출

In [17]:
result = generate_result(q_test, mfl_col, song_len, predict_plist)

In [18]:
write_json(result,f'results/result_{model_name}_rs.json')

> 평가

In [19]:
gt_fname = '../0_data/a_test.json'
rec_fname = f'results/result_{model_name}_rs.json'
arena_evaluator = ArenaEvaluator()
arena_evaluator.evaluate(gt_fname, rec_fname)

Music nDCG: 0.104066
Tag nDCG: 0.36423
Score: 0.14309
